# Caderno para construção e teste do BOT

## Loading

In [1]:
## Loader do PDF
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

path = "data\\planejamento_estrategico.pdf"

## Loaders structured and unstructured

#loader_un = UnstructuredPDFLoader(path)
loader_structured = PyMuPDFLoader(path)

pages_str = loader_structured.load()
#pages_str = loader_un.load()

## Cleaning

In [2]:
from langchain.text_splitter import CharacterTextSplitter

## Criando texto splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0
    )

## Split
texts = text_splitter.split_documents(pages_str)




In [4]:
### Testanto

texts[2].page_content

'Planejamento Estratégico\n2\nPlanejamento Estratégico da Fhemig 2024-2027\nMissão\nDetalhamento da Visão\nObjetivo da Visão\n3. Valores\nValores da Fhemig\nExemplo de Aplicação dos Valores\n4. Contextualização do Referencial Estratégico\nVariações de Solicitação de Informações\nResumo Rápido\nIndicadores do Planejamento Estratégico da Fhemig 2024-2027\nIndicadores e Metas para 2027\n1. Taxa de Ocupação Hospitalar (TOH)\n2. Tempo Médio de Permanência Hospitalar (TMP)\n3. Número de Internações\n4. Número de Cirurgias\n5. Número de Doadores Efetivos\nComo os Indicadores São Utilizados?\nResumo\nVariações de Solicitação de Informações\nObjetivos Estratégicos da Fhemig 2024-2027\n1. Perspectiva: Sociedade\n2. Perspectiva: Processos\n3. Perspectiva: Financeiro\n4. Perspectiva: Aprendizado e Crescimento\nResumo\nVariações de Solicitação de Informações\nQ&A\nMissão, Visão e Valores\nObjetivos Estratégicos\nIndicadores e Metas\nMetodologia Utilizada\nIniciativas e Projetos Estratégicos\nContex

In [5]:
## Função para remover espaço em branco

def remove_ws(d):
    text = d.page_content.replace("\n", " ")
    d.page_content = text
    return d

# Clean texts by removing whitespace from each document
texts_cleaned = [remove_ws(d) for d in texts]

In [6]:
## openai

# config.py
import os
from dotenv import load_dotenv

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path="env/.env")

# Acessar a chave da OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Retriever

In [7]:
## retriever openai

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


# uses OpenAI embeddings to build a retriever
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
# Creates the document retriever using docs and embeddings
db = FAISS.from_documents(texts_cleaned, embeddings)



# Asking the retriever to do similarity search based on Query
#query = "Foreign Aid for Lowari Road Tunnel & Access Roads Project (2nd Revised )"
#answer = db.similarity_search(query)

# Building the retriever
retriever = db.as_retriever(search_kwargs={'k': 3})


C:\Users\Acer\AppData\Local\Temp\ipykernel_23408\1278828396.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)


In [83]:
import faiss
import pickle

# Salvar o índice FAISS
faiss.write_index(db.index, 'data/faiss_index.index')

# Salvar os documentos associados (metadados)
with open('data/faiss_docs.pkl', 'wb') as f:
    pickle.dump(db.docstore._dict, f)



In [84]:
import faiss
import pickle
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Carregar o índice FAISS
index = faiss.read_index('data/faiss_index.index')

# Carregar os documentos (metadados)
with open('data/faiss_docs.pkl', 'rb') as f:
    docstore = pickle.load(f)

# Recriar o OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# Recriar o FAISS VectorStore usando o índice e os documentos
db = FAISS(embedding_function=embeddings, index=index, docstore=docstore)

# Recriar o retriever
retriever = db.as_retriever(search_kwargs={'k': 3})



TypeError: FAISS.__init__() missing 1 required positional argument: 'index_to_docstore_id'

## Chain and chats

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Função que o bot usará para responder a mensagens diretas

template = """

ara treinar o modelo no contexto do seu projeto de chatbot com RAG, considerando as melhores práticas para instruir o modelo com precisão:

plaintext
Copiar código
Você é um chatbot especializado em responder perguntas sobre o Planejamento Estratégico da Fhemig 2024-2027. Sua base de conhecimento está organizada em módulos como Missão, Visão, Valores, Objetivos Estratégicos, Indicadores e Iniciativas. Use as informações fornecidas no {context} para responder de forma assertiva, clara e objetiva. Se não encontrar informações relevantes no contexto, informe que não pode responder e sugira que o usuário refine sua pergunta.

### Contexto:
{context}

### Instruções:
1. Leia o contexto acima para obter informações necessárias para responder.
2. Responda de forma clara, usando exemplos práticos se aplicável.
3. Se a pergunta estiver ambígua ou incompleta, peça esclarecimentos.
4. Nunca invente informações que não estejam no {context}.
5. Ofereça respostas em linguagem acessível, mas técnica quando necessário.

### Formato da Resposta:
1. Comece respondendo diretamente à pergunta.
2. Inclua exemplos ou explicações adicionais para enriquecer a resposta.
3. Caso relevante, apresente conexões com outras áreas do Planejamento Estratégico.

### Exemplos de perguntas:
- Qual é a missão da Fhemig?
- Como o Planejamento Estratégico impacta a experiência do usuário?
- Qual é a meta para a Taxa de Ocupação Hospitalar?
- O que significa "fortalecer a governança corporativa" no contexto da Fhemig?

Agora, responda à pergunta fornecida pelo usuário:

PERGUNTA DO USUÁRIO: {pergunta}

"""

# Converter o template em prompt_template

prompt = ChatPromptTemplate.from_template(template)


## Configurar llm com Ollama

llm = ChatOpenAI(model="gpt-4o")

## Configurar chain

chain = chain = (
# The initial dictionary uses the retriever and user supplied query
    {"context":retriever,
     "pergunta":RunnablePassthrough()}
# Feeds that context and query into the prompt then model & lastly 
# uses the ouput parser, do query for the data.
    |  prompt  | llm | StrOutputParser()
)



## Testando!

In [13]:
input = "Quais são as metas da Fhemig?"


response = chain.invoke(input)
formatted_response = response.replace("\n", "\n\n")  # Adiciona uma linha em branco após cada nova linha

print(formatted_response)


As metas da Fhemig no Planejamento Estratégico 2024-2027 estão organizadas em objetivos estratégicos que orientam suas ações para alcançar a visão de ser reconhecida pela eficiência na gestão de serviços de saúde pública. Esses objetivos estão divididos em quatro perspectivas principais, seguindo a metodologia do Balanced Scorecard (BSC):



1. **Perspectiva Sociedade:**

   - Assegurar à sociedade serviços tempestivos e de qualidade.

   - Melhorar a experiência e satisfação do usuário.

   - Aumentar o acesso dos usuários aos serviços prestados.



2. **Perspectiva de Aprendizado e Crescimento:**

   - Criação de uma política de remuneração atrelada a resultados e produtividade.

   - Programas de saúde ocupacional para garantir a segurança e bem-estar dos colaboradores.



Para atingir essas metas, a Fhemig utiliza indicadores mensuráveis, como indicadores de qualidade do atendimento ao usuário e indicadores financeiros, para monitorar o progresso e corrigir rotas quando necessário.